In [1]:
# ! pip install pyoso

## Setup

Load environment variables, import necessary libraries, and initialize the OSO client

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()

OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

## Testing

Query the metrics table for all metric names containing '_funding_' and display them in alphabetical order

In [3]:
client.to_pandas("""
SELECT metric_name
FROM metrics_v0
WHERE metric_name LIKE '%_funding_%'
ORDER BY 1
""")

,metric_name
0,GITCOIN_DONATIONS_funding_awarded_biannually
1,GITCOIN_DONATIONS_funding_awarded_daily
2,GITCOIN_DONATIONS_funding_awarded_monthly
3,GITCOIN_DONATIONS_funding_awarded_over_all_time
4,GITCOIN_DONATIONS_funding_awarded_quarterly
5,GITCOIN_DONATIONS_funding_awarded_weekly
6,GITCOIN_DONATIONS_funding_awarded_yearly
7,GITCOIN_MATCHING_funding_awarded_biannually
8,GITCOIN_MATCHING_funding_awarded_daily
9,GITCOIN_MATCHING_funding_awarded_monthly


## Aggregate funding metrics

### By source

We currently support CSV data uploads via [oss-funding](https://github.com/opensource-observer/oss-funding) and Gitcoin Grants. We also have Open Collective deposits, but they don't show up here (yet).

In [4]:
client.to_pandas("""
SELECT
  m.metric_name,
  SUM(km.amount) AS total_amount_in_usd
FROM key_metrics_by_project_v0 AS km
JOIN metrics_v0 AS m ON km.metric_id = m.metric_id
WHERE m.metric_name LIKE '%_funding_%'
GROUP BY 1
ORDER BY 2 DESC
""")

,metric_name,total_amount_in_usd
0,OSS_FUNDING_funding_awarded_over_all_time,364887873.600968
1,GITCOIN_MATCHING_funding_awarded_over_all_time,13305117.158144
2,GITCOIN_DONATIONS_funding_awarded_over_all_time,11666103.711711


### To projects

We can also see the largest project recipients with this query.

In [5]:
client.to_pandas("""
SELECT
  p.display_name AS project_display_name,
  SUM(km.amount) AS total_amount_in_usd
FROM key_metrics_by_project_v0 AS km
JOIN metrics_v0 AS m ON km.metric_id = m.metric_id
JOIN projects_v1 AS p ON km.project_id = p.project_id
WHERE m.metric_name LIKE '%_funding_awarded_over_all_time'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")

,project_display_name,total_amount_in_usd
0,GMX,21000000.0
1,MUX Protocol,10876479.0
2,Synthetix,10022628.074157
3,Perpetual Protocol,9287212.140718
4,Gains Network,7898396.135
5,Velodrome,7895037.76024
6,Camelot,5407500.0
7,Stargate Finance,5289458.865658
8,Vertex Protocol,5250000.0
9,Radiant,4991077.0


### To projects from a specific source

In [6]:
client.to_pandas("""
SELECT
  p.display_name AS project_display_name,
  SUM(km.amount) AS total_amount_in_usd
FROM key_metrics_by_project_v0 AS km
JOIN metrics_v0 AS m ON km.metric_id = m.metric_id
JOIN projects_v1 AS p ON km.project_id = p.project_id
WHERE m.metric_name = 'GITCOIN_DONATIONS_funding_awarded_over_all_time'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")

,project_display_name,total_amount_in_usd
0,Gitcoin,1099895.038376
1,Revoke,748859.365745
2,DefiLlama,429924.507285
3,Hey,360529.24178
4,JediSwap,333277.670918
5,Dark Forest,332205.420888
6,ZigZag Exchange,210175.931949
7,ethers.js,190702.539836
8,rotki,174990.340416
9,Taho,170854.869607


### To projects from a specific source and time frame

In [7]:
client.to_pandas("""
SELECT
  p.display_name AS project_display_name,
  SUM(tm.amount) AS total_amount_in_usd
FROM timeseries_metrics_by_project_v0 AS tm
JOIN metrics_v0 AS m ON tm.metric_id = m.metric_id
JOIN projects_v1 AS p ON tm.project_id = p.project_id
WHERE m.metric_name = 'GITCOIN_DONATIONS_funding_awarded_yearly'
AND tm.sample_date < DATE '2022-01-01'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")

,project_display_name,total_amount_in_usd
0,Gitcoin,797206.330376
1,Dark Forest,297517.115925
2,ZigZag Exchange,199746.433382
3,ethers.js,129500.966707
4,Prysm Ethereum Client,128522.705766
5,rotki,122666.927997
6,ZeroPool,116795.642612
7,Lighthouse by Sigma Prime,114759.839844
8,The Tor Project,110669.738113
9,Hardhat,110539.758225


## More granular analysis

### Gitcoin

Deep dive on Gitcoin grants to a specific project

In [8]:
client.to_pandas("""
  SELECT
    time,
    round_number,
    round_name,
    event_source,
    donor_address,
    amount_in_usd
  FROM int_events__gitcoin_funding
  WHERE gitcoin_group_project_name = 'revokecash'
  ORDER BY amount_in_usd DESC
  LIMIT 10
""")

,time,round_number,round_name,event_source,donor_address,amount_in_usd
0,2023-02-09 15:58:07.078,<NA>,Gitcoin Grants,GITCOIN_DONATIONS,0x386ea3171dcc9405311fd75b316cc2a87ecadeca,617893.575
1,2023-08-29 08:32:57.000,18,Web3 Open Source Software,GITCOIN_MATCHING,<NA>,15001.1375
2,2024-11-25 14:26:59.000,22,GG22 OSS - dApps and Apps,GITCOIN_MATCHING,<NA>,14984.28125
3,2024-05-31 14:35:02.000,20,dApps & Apps,GITCOIN_MATCHING,<NA>,14979.978125
4,2023-11-29 20:18:47.000,19,Web3 Open Source Software,GITCOIN_MATCHING,<NA>,14849.591509
5,2022-08-24 00:00:00.000,15,<NA>,GITCOIN_MATCHING,<NA>,12500.0
6,2024-08-26 15:19:00.000,21,GG21: Thriving Arbitrum Summer,GITCOIN_MATCHING,<NA>,9839.680095
7,2022-08-24 00:00:00.000,15,<NA>,GITCOIN_MATCHING,<NA>,7410.488854
8,2024-05-07 10:04:49.000,20,dApps & Apps,GITCOIN_DONATIONS,0xe2a26d5174b133abc4b338df1b07295f03a4c85e,1000.42865
9,2024-05-06 17:29:47.000,20,dApps & Apps,GITCOIN_DONATIONS,0x60a06b2eee871e349331143ef173ecefd7a8ce01,537.338562


## OSS Funding

Overview of specific funders and grant pools in oss-funding data

In [9]:
client.to_pandas("""
SELECT
  from_funder_name,
  COUNT(DISTINCT grant_pool_name) AS grant_pools,
  SUM(amount) AS amount_in_usd
FROM stg_ossd__current_funding
GROUP BY 1
ORDER BY 3 DESC
""")

,from_funder_name,grant_pools,amount_in_usd
0,optimism,12,240450291.744
1,arbitrumfoundation,1,122850952.0
2,stellar,29,32989032.98
3,octant-golemfoundation,5,3965429.51329
4,dao-drops-dorgtech,1,250001.0
5,clrfund,1,83028.740386


## Funding flows

We can use this to construct a simple sankey diagram of funding flows

In [10]:
query = """
SELECT
  fp.project_id AS from_project_id,
  tp.project_id AS to_project_id,
  fp.display_name AS funder,
  tp.display_name AS project,
  SUM(e.amount) AS amount
FROM int_events_daily__funding AS e
JOIN artifacts_by_project_v1 AS fa
  ON e.from_artifact_id = fa.artifact_id
JOIN artifacts_by_project_v1 AS ta
  ON e.to_artifact_id = ta.artifact_id
JOIN projects_v1 AS fp
  ON fa.project_id = fp.project_id
JOIN projects_v1 AS tp
  ON ta.project_id = tp.project_id
GROUP BY 1,2,3,4
"""
df = client.to_pandas(query)
df.tail()

,from_project_id,to_project_id,funder,project,amount
2142,Qgbm336fY9862LN2Czg3UX04A3p7I/79Bv2M4D61DAI=,8IKXraxq1pDuQD1xaDI20cjFrel55TZ/zf6LmP69qEg=,Gitcoin,efdevcon,13.531599
2143,Qgbm336fY9862LN2Czg3UX04A3p7I/79Bv2M4D61DAI=,79HQoZtyZftibazh6Yz63aU06XODWs7b/9h4JAqPa1s=,Gitcoin,LexDAO,86499.728685
2144,5Fgf9xv3CxTV+YbSShdY9XCJs7tgW8KNwQWq9rHUEsQ=,79HQoZtyZftibazh6Yz63aU06XODWs7b/9h4JAqPa1s=,clr.fund,LexDAO,193.952856
2145,Qgbm336fY9862LN2Czg3UX04A3p7I/79Bv2M4D61DAI=,yEebFy4M1iAdb9+YQmdssSx9Qf+ZXfSVguL/JyidngI=,Gitcoin,DeFiEye,224058.115245
2146,5Fgf9xv3CxTV+YbSShdY9XCJs7tgW8KNwQWq9rHUEsQ=,JQtLQErRk0u41xS292Cg+s3cRr8LaD5lQ2kME/Syp2Q=,clr.fund,Asilo Digital,703.639308
